In [ ]:
import os
import json
import pandas as pd
from datetime import datetime

import pymongo
from pymongo import MongoClient

In [ ]:
client = MongoClient('localhost', 27017)
db = client.DataManagement
treni_regionali = db.TreniRegionali
altri_treni = db.AltriTreni

In [ ]:
percorso= 'H:/.shortcut-targets-by-id/1ZqofLGs2bjcl1THsk3r-ac-YPkGL5QH8/Progetto Data Man/File'
staz = pd.read_csv('H:/.shortcut-targets-by-id/1ZqofLGs2bjcl1THsk3r-ac-YPkGL5QH8/Progetto Data Man/Lista stazioni/stazioni_finale.csv')

In [ ]:
for directory in os.listdir(percorso):
    print(directory)
    regionali = list()
    nonregionali = list()
    if(directory[0] != "." and directory != "desktop.ini"):
        for file in os.listdir(percorso +  "/"  + directory + "/"):
            if file[0] != "." and file != "desktop.ini":
                print(file) # per individuare eventuale errori
                _json = json.loads(open(percorso + "/" + directory + "/" + file).read()) 
                treno = _json[0]
                numero = str(treno["numeroTreno"])
                categoria = treno["categoria"]
                ritardo_finale = treno["ritardo"]
                fermate = treno["fermate"]
                idDestinazione = treno["idDestinazione"] 
                idOrigine = treno["idOrigine"] 
                origine = treno["origine"]
                destinazione = treno["destinazione"]

                if treno["tipoTreno"] == "ST":
                    if treno["categoria"] == "REG":
                        regionali.append({"numero_treno": numero, 
                                        "categoria": categoria,
                                        "stato": "cancellato",
                                        "origine":origine,
                                        "id_origine": idOrigine,
                                        "destinazione":destinazione,
                                        "id_destinazione": idDestinazione, 
                                        "data": datetime.strptime(directory,"%d-%m-%Y").strftime('%d-%m-%Y')
                        })
                    else:
                        nonregionali.append({"numero_treno": numero, 
                                        "categoria": categoria,
                                        "stato": "cancellato",
                                        "origine":origine,
                                        "id_origine": idOrigine,
                                        "destinazione":destinazione,
                                        "id_destinazione": idDestinazione, 
                                        "data": datetime.strptime(directory,"%d-%m-%Y").strftime('%d-%m-%Y')
                        })

                else:
                    if fermate != None:
                        if len(fermate) != 0:
                            fer = []
                            for s in fermate:
                                dati_staz = staz[staz["id"] == s["id"]][["nome_stazione","binari_finale"]]

                                if s["tipoFermata"] == "P":
                                    if s["partenzaReale"] == None or s["ritardoPartenza"] < -5:
                                        _ritardo_partenza = None
                                    else:
                                        _ritardo_partenza = s["ritardoPartenza"]

                                    if s["partenza_teorica"] and s["partenza_teorica"] > 1640991600000 and s["partenza_teorica"] < 1672527599000:
                                        _partenza_teorica = datetime.fromtimestamp(s["partenza_teorica"]/1000).strftime('%H:%M')
                                    else:
                                        _partenza_teorica = s["partenza_teorica"]
                                    
                                    if s["partenzaReale"] and s["partenzaReale"] > 1640991600000 and s["partenzaReale"] < 1672527599000:
                                        _partenza_reale = datetime.fromtimestamp(s["partenzaReale"]/1000).strftime('%H:%M')
                                    else:
                                        _partenza_reale = s["partenzaReale"]
                                    
                                    fer.append({"tipo_fermata": s["tipoFermata"],
                                                "nome_stazione": s["stazione"],
                                                "id_stazione": s["id"], 
                                                "binari_tot_stazione": dati_staz["binari_finale"].to_string(index=False),
                                                "partenza_teorica": _partenza_teorica,
                                                "partenza_reale": _partenza_reale, 
                                                "ritardo_partenza": _ritardo_partenza                                                
                                    })
                                
                                elif s["tipoFermata"] == "F":
                                    if s["partenzaReale"] == None or s["ritardoPartenza"] < -5:
                                        _ritardo_partenza = None
                                    else:
                                        _ritardo_partenza = s["ritardoPartenza"]
                                    
                                    if s["arrivoReale"] == None or s["ritardoArrivo"] < -5:
                                        _ritardo_arrivo = None
                                    else:
                                        _ritardo_arrivo = s["ritardoArrivo"]

                                    if s["partenza_teorica"] and s["partenza_teorica"] > 1640991600000 and s["partenza_teorica"] < 1672527599000:
                                        _partenza_teorica = datetime.fromtimestamp(s["partenza_teorica"]/1000).strftime('%H:%M')
                                    else:
                                        _partenza_teorica = s["partenza_teorica"]
                                    
                                    if s["partenzaReale"] and s["partenzaReale"] > 1640991600000 and s["partenzaReale"] < 1672527599000:
                                        _partenza_reale = datetime.fromtimestamp(s["partenzaReale"]/1000).strftime('%H:%M')
                                    else:
                                        _partenza_reale = s["partenzaReale"]

                                    if s["arrivo_teorico"] and s["arrivo_teorico"] > 1640991600000 and s["arrivo_teorico"] < 1672527599000:
                                        _arrivo_teorico = datetime.fromtimestamp(s["arrivo_teorico"]/1000).strftime('%H:%M')
                                    else:
                                        _arrivo_teorico = s["arrivo_teorico"]
                                    
                                    if s["arrivoReale"] and s["arrivoReale"] > 1640991600000 and s["arrivoReale"] < 1672527599000:
                                        _arrivo_reale = datetime.fromtimestamp(s["arrivoReale"]/1000).strftime('%H:%M')
                                    else:
                                        _arrivo_reale = s["arrivoReale"]
                                        
                                    fer.append({"tipo_fermata": s["tipoFermata"],
                                                "nome_stazione": s["stazione"],
                                                "id_stazione": s["id"], 
                                                "binari_tot_stazione": dati_staz["binari_finale"].to_string(index=False),
                                                "arrivo_teorico": _arrivo_teorico, 
                                                "arrivo_reale": _arrivo_reale, 
                                                "ritardo_arrivo": _ritardo_arrivo,
                                                "partenza_teorica": _partenza_teorica, 
                                                "partenza_reale": _partenza_reale, 
                                                "ritardo_partenza": _ritardo_partenza
                                    })
                                elif s["tipoFermata"] == "A":
                                    if s["arrivoReale"] == None or s["ritardoArrivo"] < -5:
                                        _ritardo_arrivo = None
                                    else:
                                        _ritardo_arrivo = s["ritardoArrivo"]
                                    
                                    if s["arrivo_teorico"] and s["arrivo_teorico"] > 1640991600000 and s["arrivo_teorico"] < 1672527599000:
                                        _arrivo_teorico = datetime.fromtimestamp(s["arrivo_teorico"]/1000).strftime('%H:%M')
                                    else:
                                        _arrivo_teorico = s["arrivo_teorico"]
                                    
                                    if s["arrivoReale"] and s["arrivoReale"] > 1640991600000 and s["arrivoReale"] < 1672527599000:
                                        _arrivo_reale = datetime.fromtimestamp(s["arrivoReale"]/1000).strftime('%H:%M')
                                    else:
                                        _arrivo_reale = s["arrivoReale"]

                                        
                                    fer.append({"tipo_fermata": s["tipoFermata"],
                                                "nome_stazione": s["stazione"],
                                                "id_stazione": s["id"], 
                                                "binari_tot_stazione": dati_staz["binari_finale"].to_string(index=False),
                                                "arrivo_teorico": _arrivo_teorico, 
                                                "arrivo_reale": _arrivo_reale,
                                                "ritardo_arrivo": _ritardo_arrivo,
                                    })
                            new_fermate = fer                     
                    else:
                        new_fermate = "non rilevate"
                    
                    if treno["tipoTreno"] != "PG":
                        if treno["categoria"] == "REG":
                            regionali.append({"numero_treno": numero,
                                        "categoria": categoria,
                                        "stato": "parzialmente soppresso",
                                        "origine_teorica": treno["origineZero"],
                                        "origine": origine,
                                        "id_origine": idOrigine, 
                                        "destinazione_teorica": treno["destinazioneZero"],
                                        "destinazione": destinazione,
                                        "id_destinazione": idDestinazione, 
                                        "data": datetime.strptime(directory,"%d-%m-%Y").strftime('%d-%m-%Y'),
                                        "ritardo_finale": ritardo_finale,
                                        "fermate": new_fermate
                            })
                        else:
                            nonregionali.append({"numero_treno": numero,
                                        "categoria": categoria,
                                        "stato": "parzialmente soppresso",
                                        "origine_teorica": treno["origineZero"],
                                        "origine": origine,
                                        "id_origine": idOrigine, 
                                        "destinazione_teorica": treno["destinazioneZero"],
                                        "destinazione": destinazione,
                                        "id_destinazione": idDestinazione, 
                                        "data": datetime.strptime(directory,"%d-%m-%Y").strftime('%d-%m-%Y'),
                                        "ritardo_finale": ritardo_finale,
                                        "fermate": new_fermate
                            })

                    else:
                        if treno["categoria"] == "REG":
                            regionali.append({"numero_treno": numero,
                                        "categoria": categoria,
                                        "stato": "regolare",
                                        "origine":origine,
                                        "id_origine": idOrigine, 
                                        "destinazione": destinazione,
                                        "id_destinazione": idDestinazione,
                                        "data": datetime.strptime(directory,"%d-%m-%Y").strftime('%d-%m-%Y'),
                                        "ritardo_finale": ritardo_finale,
                                        "fermate": new_fermate
                            })
                        else:
                            nonregionali.append({"numero_treno": numero,
                                        "categoria": categoria,
                                        "stato": "regolare",
                                        "origine":origine,
                                        "id_origine": idOrigine, 
                                        "destinazione": destinazione,
                                        "id_destinazione": idDestinazione,
                                        "data": datetime.strptime(directory,"%d-%m-%Y").strftime('%d-%m-%Y'),
                                        "ritardo_finale": ritardo_finale,
                                        "fermate": new_fermate
                            })
        x = treni_regionali.insert_many(regionali)
        y = altri_treni.insert_many(nonregionali)
        print(x)
        print(y)